# 주요 질병 사망자 및 보건 환경 비교 분석
## - 6 - 종합 및 결론
(Comparative Analysis of Major Disease Deaths and Health Environment)
## 1. 소개
국가별 의료 관련 데이터들을 이용해, 국가별, 연간 질병 사망자 수와 의료 인프라 간의 관계를 분석해보고 시각화를 해본 프로젝트

## 2. 목표
- 각 데이터의 EDA와 시각화
- 비슷한 분류의 데이터를 하나로 합치기
- 의료 인프라와 질병 사망자 수 관계 찾기 및 분석

## 3. 데이터 셋
- [KOSIS 10만명당 주요사망원인별 사망률](https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_2KAAC13&conn_path=I2)
- [KOSIS 의료종사자수](https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_2KAAC01&conn_path=I2)
- [KOSIS 보건관련 지출비](https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_2KAAC09&conn_path=I2)
- [KOSIS 보건서비스 지출비](https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_2KAAC14&conn_path=I2)
- [KOSIS 의약품 판매](https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_2KAAC25&conn_path=I2)
- [KOSIS 의약품 소비](https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_2KAAC24&conn_path=I2)
- [KOSIS 공공사회복지 지출](https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_2KAAD21&conn_path=I2)
- [geojson 데이터](https://github.com/dr5hn/countries-states-cities-database/blob/master/countries.json)

## 4. 구현

#### 라이브러리

In [1]:
import glob
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import cufflinks as cf
import folium
from folium.plugins import MarkerCluster
import json

#### 한글 폰트 설정

In [2]:
from IPython.display import set_matplotlib_formats

def get_font_family():
    import platform
    system_name = platform.system()

    if system_name == "Darwin" :
        font_family = "AppleGothic"
    elif system_name == "Windows":
        font_family = "Malgun Gothic"
    else:
        !apt-get install fonts-nanum -qq  > /dev/null
        !fc-cache -fv

        import matplotlib as mpl
        mpl.font_manager._rebuild()
        findfont = mpl.font_manager.fontManager.findfont
        mpl.font_manager.findfont = findfont
        mpl.backends.backend_agg.findfont = findfont
        
        font_family = "NanumBarunGothic"
    return font_family

get_font_family()

plt.style.use("ggplot")

font_family = get_font_family()
plt.rc("font", family=font_family)
plt.rc("axes", unicode_minus=False)

set_matplotlib_formats("retina")

C:\Users\spec3\AppData\Local\Temp\ipykernel_8816\2328754805.py:32: DeprecationWarning:

`set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`



In [3]:
# 모든 데이터 공통 국가
country_intersection = ['룩셈부르크',  '네덜란드',  '영국',  '이탈리아',  '캐나다',  '오스트레일리아',  '한국',  '일본',  '스페인',  '헝가리',  '독일',  '에스토니아',  '그리스',  '슬로바키아',  '핀란드',  '벨기에',  '체코',  '슬로베니아',  '프랑스',  '스웨덴',  '노르웨이',  '뉴질랜드',  '라트비아',  '덴마크',  '오스트리아',  '포르투갈',  '아일랜드',  '아이슬란드']

#### Data Load
각 파트별로 EDA 및 시각화를 진행한 후, 최종적으로 전처리된 파일은 6개입니다.  
**종합 및 결론**에서 사용하는 데이터 파일은 다음과 같이 총 8개 입니다.
- [KOSIS 10만명당 주요사망원인별 사망률](https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_2KAAC13&vw_cd=MT_RTITLE&list_id=101_003_E&seqNo=&lang_mode=ko&language=kor&obj_var_id=&itm_id=&conn_path=MT_RTITLE) → `df_death_rate`
- [KOSIS 의료종사자수](https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_2KAAC13&vw_cd=MT_RTITLE&list_id=101_003_E&seqNo=&lang_mode=ko&language=kor&obj_var_id=&itm_id=&conn_path=MT_RTITLE) → `df_Nmw`
- [KOSIS 보건관련 지출비](https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_2KAAC13&vw_cd=MT_RTITLE&list_id=101_003_E&seqNo=&lang_mode=ko&language=kor&obj_var_id=&itm_id=&conn_path=MT_RTITLE) → `df_service`
- [KOSIS 보건서비스 지출비](https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_2KAAC13&vw_cd=MT_RTITLE&list_id=101_003_E&seqNo=&lang_mode=ko&language=kor&obj_var_id=&itm_id=&conn_path=MT_RTITLE) → `df_service_common`
- [KOSIS 의약품 판매](https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_2KAAC13&vw_cd=MT_RTITLE&list_id=101_003_E&seqNo=&lang_mode=ko&language=kor&obj_var_id=&itm_id=&conn_path=MT_RTITLE) → `df_medicion`
- [KOSIS 의약품 소비](https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_2KAAC13&vw_cd=MT_RTITLE&list_id=101_003_E&seqNo=&lang_mode=ko&language=kor&obj_var_id=&itm_id=&conn_path=MT_RTITLE) → `df_medicion`
- [KOSIS 공공사회복지 지출](https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_2KAAC13&vw_cd=MT_RTITLE&list_id=101_003_E&seqNo=&lang_mode=ko&language=kor&obj_var_id=&itm_id=&conn_path=MT_RTITLE) → `df_welfare`
- [geojson 데이터](https://github.com/dr5hn/countries-states-cities-database/blob/master/countries.json) → `countries.geo.edited.json`, `countries.json`

In [41]:
df_death_rate = pd.read_csv("data/pre_df/df_death_rate.csv")
df_Nmw = pd.read_csv("data/pre_df/df_Nmw.csv")
df_service = pd.read_csv("data/pre_df/df_service.csv")
df_service_common = pd.read_csv("data/pre_df/df_service_common.csv")
df_medicion = pd.read_csv("data/pre_df/df_medicion.csv")
df_welfare = pd.read_csv("data/pre_df/df_welfare.csv")

In [9]:
geo_poly = open("data/countries.geo.edited.json", encoding="utf-8")
gp = open("data/countries.json", encoding="utf-8")
geo_point = pd.json_normalize(json.load(gp))
geo_poly.close()
gp.close()

In [42]:
print(f"df_death_rate: {df_death_rate.shape}\ndf_Nmw: {df_Nmw.shape}\ndf_service: {df_service.shape}\ndf_service_common: {df_service_common.shape}\ndf_medicion: {df_medicion.shape}\ndf_welfare: {df_welfare.shape}")

df_death_rate: (18507, 8)
df_Nmw: (1829, 7)
df_service: (3192, 5)
df_service_common: (3788, 11)
df_medicion: (4724, 5)
df_welfare: (893, 9)


In [43]:
df_death_rate.head()

,국가,질병명,연도,성별,사망률,국가영문,위도,경도
0,한국,특정감염성 및 기생충성질환,1998,남자,24.6,korea,37.0,127.5
1,한국,특정감염성 및 기생충성질환,1998,여자,8.8,korea,37.0,127.5
2,한국,호흡기결핵,1998,남자,15.8,korea,37.0,127.5
3,한국,호흡기결핵,1998,여자,4.1,korea,37.0,127.5
4,한국,악성신생물(암),1998,남자,196.2,korea,37.0,127.5


In [44]:
df_Nmw.head()

,국가,연도,직업,수,국가영문,위도,경도
0,한국,1998,의사,58684.0,korea,37.0,127.5
1,한국,1998,치과의사,12875.0,korea,37.0,127.5
2,한국,1998,간호사,128427.0,korea,37.0,127.5
3,한국,1999,의사,61182.0,korea,37.0,127.5
4,한국,1999,치과의사,14058.0,korea,37.0,127.5


In [51]:
df_service.sample(5)

,국가별,항목,데이터값,연도,국가영문
1848,한국,경상(GDP 대비%),6.008444,2011,South Korea
1252,헝가리,1인당 보건지출(PPP)(US$),1372.693970,2007,Hungary
100,라트비아,1인당 보건지출(PPP)(US$),434.380096,2000,Latvia
2740,핀란드,1인당 보건지출(PPP)(US$),4211.662109,2016,Finland
2522,한국,본인부담(경상보건비 대비%),33.599072,2015,South Korea


In [52]:
df_service_common.sample(5)

,대륙,국가,공통국가여부,항목구분,연도,값(백만 자국화폐),전년대비증감,key_currency_x,key_currency_ext,환율(자국화폐단위/$),서비스비용(백만$)
675,유럽,벨기에,공통,보건 보조 서비스,2012,1974.0,210.0,2012벨기에,2012덴마크,5.79,340.93
1351,유럽,프랑스,공통,보건 당기 지출,2013,241706.0,5725.0,2013프랑스,2013덴마크,5.62,43008.19
2578,유럽,네덜란드,공통,장기 치료,2009,16131.0,698.0,2009네덜란드,2009덴마크,5.36,3009.51
3435,유럽,스웨덴,공통,장기 치료,2007,21145.0,862.0,2007스웨덴,2007스웨덴,6.76,3127.96
805,유럽,체코,공통,보건 보조 서비스,2009,15499.0,1056.0,2009체코,2009체코,19.06,813.17


In [50]:
df_welfare.sample(10)

,대륙,국가,공통국가여부,연도,값(% of gdp),전년대비증감,key_gdp,GDP(10억$),복지비용(10억$)
740,유럽,슬로베니아,공통,2017,21.5,-0.8,슬로베니아2017,48.5,1042.75
98,아시아,터키,해당아님,2010,12.3,-0.7,터키2010,777.0,9557.10
824,유럽,영국,공통,2014,21.9,-0.5,영국2014,3065.5,67134.45
398,유럽,프랑스,공통,2004,28.9,0.1,프랑스2004,2115.7,61143.73
469,유럽,헝가리,공통,2010,23.0,-0.3,헝가리2010,131.9,3033.70
159,북아메리카,멕시코,해당아님,2006,6.3,0.1,멕시코2006,975.4,6145.02
385,유럽,핀란드,공통,2013,29.4,1.1,핀란드2013,271.3,7976.22
435,유럽,독일,공통,2019,25.9,0.5,독일2019,3861.1,100002.49
521,유럽,아일랜드,공통,2018,13.6,-0.6,아일랜드2018,386.0,5249.60
9,아시아,소계,해당아님,2007,NaN,NaN,소계2007,NaN,NaN


In [48]:
df_medicion.head()

,국가,연도,의약품,의약품소비량,의약품판매량
0,한국,2008,소화관 및 신진대사,263.6,15821.1
1,한국,2008,제산제,2.2,15821.1
2,한국,2008,소화성궤양 및 위장용 약물,33.1,15821.1
3,한국,2008,당뇨 약물,59.8,15821.1
4,한국,2008,심장 배당체,1.5,15821.1
